In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pystan
import scipy
import math

In [17]:
df = pd.read_excel("/Users/apple/Downloads/userVisits.xlsx")
df.head()

,user_id,total_visits,weeklyVisits,Interval_visits_int
0,1332384,70,"0,0,0,1,2,1,1,0,0,2,1,0,0,2,1,0,0,0,1,1,2,1,0,...","[3, 0, 0, 0, 2, 0, 2, 0, 3, 0, 0, 0, 1, 4, 2, ..."
1,1332636,4,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","[62, 12, 1, 28]"
2,1334543,4,"1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","[6, 0]"
3,1336195,1,"0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",[113]
4,1339702,16,"1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...","[110, 14, 29, 7, 29, 2, 2, 1, 0, 2, 4, 1, 1, 1..."


In [18]:
#df = df.iloc[df.index <= 200]

In [19]:
visits = df['weeklyVisits'].tolist()

In [20]:
visits_int = list(list(int(a) for a in (str(r).split(',')))for r in visits)

In [21]:
len(visits_int)

25000

In [22]:
len(visits_int[10])

113

In [ ]:
visitsg_code = """
data {
    int<lower=0> J;// number of users
    int<lower=0> G;// number of groups
    int<lower=0> W;// number of weeks
    //int<lower=0> K[G];// number of users in a group
    int y[J,W]; // estimated number of visits next week
    //real<lower=0> sigma[J]; // s.e. of visits estimates
}
parameters {
    //real mu;
    real<lower=0,upper=1> tau[G,J];
    
    real<lower=0> eta[J];
    real <lower=0, upper=10> etag[G];
    
    //real sigma;
}
transformed parameters {
     real <lower=0> theta[J];
     //real <lower =0> K[G];
     real <lower =0, upper =1> alpha[G,J];
     real<lower=0> gCon[J];
     real <lower=0,upper=1> count;
     
     for(j in 1:J)
     gCon[j]=0;
     
     count = 0;
     for (j in 1:J){
     for ( g in 1:G){
     alpha[g,j] = tau[g,j];
     count = count + alpha[g,j];
     }
     for(g in 1:G){
     alpha[g,j]=alpha[g,j]/count;
     
     }
     count =0;
     }
     
     
     
     for (g in 1:G){
     
     for (j in 1:J){
     gCon[j]= gCon[j] + etag[g]*alpha[g,j];
     }
     }
     
     for (j in 1:J)
     theta[j]= eta[j];

}
model {
    
    for(g in 1:G){
    //etag[g] ~ gamma(1.0,2.0);
    for (j in 1:J){
    tau[g,j] ~ beta (2,2);
    }
    }
   
    for(j in 1:J){
    eta[j] ~ gamma(gCon[j],gCon[j]);
    }
    for (j in 1:J){
    for(w in 1:W){
    y[j,w] ~ poisson(theta[j]);
    }
 }
}
"""



visitsg_dat = {'J': len(visits_int),
              'W':113,
              'G':5,
              
              'y': visits_int
               }



sm = pystan.StanModel(model_code=visitsg_code)
fit = sm.sampling(data=visitsg_dat, iter=1000, chains=4)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_7d3c5f8d7ef0af265e8c7ecf3804fbf8 NOW.


In [13]:
fit


Inference for Stan model: anon_model_7d3c5f8d7ef0af265e8c7ecf3804fbf8.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

               mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
tau[0,0]        0.5  4.9e-3   0.22    0.1   0.33    0.5   0.67   0.91   2000    1.0
tau[1,0]        0.5  5.0e-3   0.22    0.1   0.33    0.5   0.68   0.91   2000    1.0
tau[2,0]        0.5  5.1e-3   0.23   0.09   0.33   0.51   0.68   0.91   2000    1.0
tau[3,0]        0.5  5.1e-3   0.23   0.09   0.32   0.51   0.68   0.91   2000    1.0
tau[4,0]        0.5  5.1e-3   0.23   0.09   0.32    0.5   0.67    0.9   2000    1.0
tau[0,1]        0.5  5.0e-3   0.22   0.08   0.34   0.51   0.66   0.92   2000    1.0
tau[1,1]        0.5  5.1e-3   0.23   0.09   0.32    0.5   0.69   0.91   2000    1.0
tau[2,1]        0.5  5.0e-3   0.22    0.1   0.33    0.5   0.68    0.9   2000    1.0
tau[3,1]        0.5  4.9e-3   0.22    0.1   0.33   0.49  

In [ ]:
fit.